<a href="https://colab.research.google.com/github/Brian6146/TourismModel/blob/main/TourismModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader ,random_split
from torchvision import datasets, transforms
from collections import Counter
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from torchinfo import summary

In [ ]:
if torch.cuda.is_available():
  device="cuda"
elif torch.backends.mps.is_available():
  device="mps"
else:
  device="cpu"

print(f"device is running on {device}")

In [ ]:
train_dir=os.path.join("train")
classes=os.listdir(train_dir)
print(classes)

#ConvertToRGB
class ConvertToRGB(object):
  def __call__(self,img):
    if img != "RGB":
      img=img.convert("RGB")
    return img
#Build transformer pipeline
transform= transforms.Compose([
    ConvertToRGB(),
    transforms.Resize((224,224)),
    transforms.ToTensor()

    ])
#Test if transformer pipeline is working
sample_file=os.path.join("train","cheetah","Image_1.jpg")
image=Image.open(sample_file)
transformed_img=transform(image)
print(transformed_img.shape)

dataset=datasets.ImageFolder(train_dir,transform=transform)
print("image size",dataset[0][0].shape)
print("Label",dataset[0][1])

In [ ]:
dataset=datasets.ImageFolder(train_dir,transform=transform)
print("image size",dataset[0][0].shape)
print("Label",dataset[0][1])

#get the counts ,the keys are the class indices not the names
counts =Counter(x[1] for x in tqdm(dataset))
print("The counts dictionary:", counts)
#This dictionary maps class names to their indices
print("The class_to_idx dictionary",dataset.class_to_idx)
#Reverse the mapping
idx_to_class = {idx:class_name for class_name,idx in dataset.class_to_idx.items()}

class_distribution={idx_to_class[idx]:count for idx,count in counts.items()}
print("Class distribution:",class_distribution)

In [ ]:
#transform data
batch_size=32
dataset_loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)

#get one batch
first_batch=next(iter(dataset_loader))
print("Batch shape",first_batch[0].shape)
print("Label shape",first_batch[1].shape)

In [ ]:
#transform data
batch_size=32
dataset_loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)

#get one batch
first_batch=next(iter(dataset_loader))
print("Batch shape",first_batch[0].shape)
print("Label shape",first_batch[1].shape)

In [ ]:
transform_norm=transforms.Compose([
    ConvertToRGB(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

norm_dataset=datasets.ImageFolder(root=train_dir,transform=transform_norm)
print("Image size",norm_dataset[0][0].shape)
print("Label",norm_dataset[0][1])


In [ ]:
g=torch.Generator()
g.manual_seed(42)

train_dataset,val_dataset=random_split(norm_dataset,[0.8,0.2],generator=g)

print("Length of train dataset",len(train_dataset))
print("Length of val dataset",len(val_dataset))

train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=batch_size)

In [ ]:
model=torch.nn.Sequential(
    nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),

    nn.Flatten(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=64*56*56,out_features=128),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=128,out_features=4)
)

batch_size=32
height=width=224

summary(model,input_size=(batch_size,3,height,width))


In [ ]:
#prepare the model for training
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
device="cpu"
model.to(device)

In [ ]:
def train_epoch(model, optimizer, loss_fn, data_loader, device="cpu"):
    training_loss = 0.0
    model.train()

    # Iterate over all batches in the training set to complete one epoch
    for inputs, targets in tqdm(data_loader, desc="Training", leave=False):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        targets = targets.to(device)

        output = model(inputs)
        loss = loss_fn(output, targets)

        loss.backward()
        optimizer.step()
        training_loss += loss.data.item() * inputs.size(0)

    return training_loss / len(data_loader.dataset)



def predict(model, data_loader, device="cpu"):
    all_probs = torch.tensor([]).to(device)

    model.eval()
    with torch.no_grad():
        for inputs, targets in tqdm(data_loader, desc="Predicting", leave=False):
            inputs = inputs.to(device)
            output = model(inputs)
            probs = torch.nn.functional.softmax(output, dim=1)
            all_probs = torch.cat((all_probs, probs), dim=0)

    return all_probs



def score(model, data_loader, loss_fn, device="cpu"):
    total_loss = 0
    total_correct = 0

    model.eval()
    with torch.no_grad():
        for inputs, targets in tqdm(data_loader, desc="Scoring", leave=False):
            inputs = inputs.to(device)
            output = model(inputs)

            targets = targets.to(device)
            loss = loss_fn(output, targets)
            total_loss += loss.data.item() * inputs.size(0)

            correct = torch.eq(torch.argmax(output, dim=1), targets)
            total_correct += torch.sum(correct).item()
    average_loss = total_loss / len(data_loader.dataset)
    accuracy = total_correct / len(data_loader.dataset)
    return average_loss, accuracy



def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs + 1):
        training_loss = train_epoch(model, optimizer, loss_fn, train_loader, device)

        # Test on validation set
        validation_loss, validation_accuracy = score(model, val_loader, loss_fn, device)

        print(
            f"Epoch: {epoch}, Training Loss: {training_loss:.2f}, "
            f"Validation Loss: {validation_loss:.2f}, Validation accuracy = {validation_accuracy:.2f}"
        )

In [ ]:
train(model,optimizer,loss_fn,train_loader,val_loader,epochs=20,device=device)

In [ ]:
#Evaluate model performance
#compute the probabilities for each validation image
probabilities=predict(model,val_loader,device)
predictions=torch.argmax(probabilities,dim=1)
print("Number of predictions",predictions.shape)

In [ ]:
#Create a confusion matrix for the predictions on the validation set
targets=[]
for _,labels in tqdm(val_loader):
  targets.extend(labels.tolist())

fig,ax=plt.subplots(figsize=(10,6))

cm=confusion_matrix(targets,predictions.cpu())

#Get the class names
classes=os.listdir(train_dir)

#Display the confusion matrix
disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=classes)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45,ax=ax)



In [ ]:
test_dir=os.path.join("test")
test_dataset=datasets.ImageFolder(test_dir,transform_norm)
test_loader=DataLoader(test_dataset,batch_size=10)
print("Number of test images",len(test_dataset))

In [ ]:
#predict the probabilities for each test image
test_probabilities=predict(model,test_loader,device)
test_predictions=torch.argmax(probabilities,dim=1)
print("Number of predictions",predictions.shape)

test_classes=[classes[i] for i in test_predictions]
print("Number of class predictions:",len(test_classes))

In [ ]:
import matplotlib.pyplot as plt
import random

#Sample 12 random indices from the test dataset
random_indices=random.sample(range(len(test_loader.dataset.samples)),12)

fig,axes=plt.subplots(4,3,figsize=(20,10))

#Iterates over the sampled indices and plots the corresponding images
for ax,idx in zip(axes.flatten(),random_indices):
  img_path=test_loader.dataset.samples[idx][0]
  img=Image.open(img_path)

  #Displays images on the axis
  ax.imshow(img)
  ax.axis("off")

  #Gets the predicted class for the image
  predicted_class=test_classes[idx]

  #Sets the title of the axis to the predicted class
  ax.set_title(predicted_class)



plt.tight_layout()